In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/ifajcik/efficientqa_pruned/efficientQA_submission'

In [2]:
! ls | grep 'NQ' 

NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_abstractive_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonlfused_w_abs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_gen_reranked_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_preprocessed_queryenc_wikipassages_NQ_open_val_<class 'transformers.tokenization_roberta.RobertaTokenizer'>.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_ranked_outputs.jsonl
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl_fuseindecoder_preprocessed_for_C25_t5-base_L512.json


In [3]:
!ls "/mnt/athena19/opt/effqa_full/efficientQA_submission" | grep 'NQ' 

NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_abstractive_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonlfused_w_abs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_gen_reranked_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_preprocessed_queryenc_wikipassages_NQ_open_val_<class 'transformers.tokenization_roberta.RobertaTokenizer'>.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_ranked_outputs.jsonl
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl_fuseindecoder_preprocessed_for_C25_t5-base_L512.json


In [4]:
import json
import jsonlines

prefix = ""
#prefix = "/mnt/athena19/opt/effqa_full/efficientQA_submission/"

#extractive_reader_output_f = prefix + "NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reader_outputs.json"
extractive_reader_output_f = prefix + "NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused.json"
ranker_predictions=prefix +"NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_ranked_outputs.jsonl"
reranker_predictions=prefix +"NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl"


gt_file = "efficientqa_eval/NQ-open.efficientqa.dev.1.1.jsonl"

with open(extractive_reader_output_f, encoding="utf8") as f:
    extractive_reader_predictions = json.load(f)
with jsonlines.open(ranker_predictions) as rf:    
    ranked_predictions = list(rf)
with jsonlines.open(reranker_predictions) as rrf:    
    reranked_predictions = list(rrf)

inference_type="hardmax" # hardmax / marginal

MAX_ANS = 50
    
import jsonlines
correct = {x['question']:x['answer'] for x in list(jsonlines.open(gt_file))}

In [5]:
print(f"Maximum number of extractive answers: {len(extractive_reader_predictions['0']['answers'])}")

Maximum number of extractive answers: 50


In [6]:
print(len(extractive_reader_predictions))
print(len(ranked_predictions))
print(len(reranked_predictions))

1800
1800
1800


In [7]:
ranked_predictions[0].keys()

dict_keys(['id', 'question', 'predicted_indices', 'predicted_scores'])

In [8]:
import fit_qa.scripts.common.evaluate_predictions as eval_utils
from fit_qa.scripts.common.evaluate_predictions import exact_match_score

labels = []
ext_scores, ranker_scores, reranker_scores = [], [], []

assert len(extractive_reader_predictions.values()) == len(ranked_predictions) == len(reranked_predictions)
for e_p, ranked_p, reranked_p in zip(extractive_reader_predictions.values(),ranked_predictions,reranked_predictions):
    assert e_p['raw_question'] == ranked_p['question'] == reranked_p['question']
    correct_answers = correct[e_p['raw_question']]
    
    # Get ground truth rank, if there is gt
    gt_rank = -1 
    for i, a in enumerate(e_p['answers']):
        if eval_utils.metric_max_over_ground_truths(
            metric_fn=exact_match_score, prediction=a, ground_truths=correct_answers):
            gt_rank = i
            break
    if gt_rank <0:
        continue
    assert len(e_p['reader_scores'])==MAX_ANS
    labels.append(gt_rank)
    
    # Get ranker and reranker scores for each answer's passage
    answer_passages = e_p['passages']
    
   
    rs = [ranked_p['predicted_scores'][ranked_p['predicted_indices'].index(passage_idx)] 
                     for passage_idx in answer_passages]
    rrs = [reranked_p['predicted_scores'][reranked_p['predicted_indices'].index(passage_idx)] 
                     for passage_idx in answer_passages]
    ext_scores.append(e_p['reader_scores'])
    ranker_scores.append(rs)
    reranker_scores.append(rrs)
    

In [9]:
ext_scores_test, ranker_scores_test, reranker_scores_test = [], [], []

for e_p, ranked_p, reranked_p in zip(extractive_reader_predictions.values(),ranked_predictions,reranked_predictions):
    assert e_p['raw_question'] == ranked_p['question'] == reranked_p['question']
       
    # Get ranker and reranker scores for each answer's passage
    answer_passages = e_p['passages']
    
   
    rs = [ranked_p['predicted_scores'][ranked_p['predicted_indices'].index(passage_idx)] 
                     for passage_idx in answer_passages]
    rrs = [reranked_p['predicted_scores'][reranked_p['predicted_indices'].index(passage_idx)] 
                     for passage_idx in answer_passages]
    ext_scores_test.append(e_p['reader_scores'])
    ranker_scores_test.append(rs)
    reranker_scores_test.append(rrs)

In [10]:
import torch
labels = torch.LongTensor(labels)
X = torch.FloatTensor(ext_scores)
Y = torch.FloatTensor(ranker_scores)
Z = torch.FloatTensor(reranker_scores)
X_test = torch.FloatTensor(ext_scores_test)
Y_test = torch.FloatTensor(ranker_scores_test)
Z_test = torch.FloatTensor(reranker_scores_test)

In [11]:
X.shape, Y.shape, Z.shape , X_test.shape, Y_test.shape, Z_test.shape

(torch.Size([1240, 50]),
 torch.Size([1240, 50]),
 torch.Size([1240, 50]),
 torch.Size([1800, 50]),
 torch.Size([1800, 50]),
 torch.Size([1800, 50]))

In [24]:
import torch
class ConstrainedLR(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        self.b = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        self.c = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        self.bias = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        #top-k prior for each position
        #self.bias = torch.nn.Parameter(torch.randn((50), requires_grad=True))
        
    def forward(self,X,Y,Z):
        r = X*self.a +  Y*self.b + Z * self.c + self.bias 
        return r

In [13]:
def evaluate(preds, gts):
    assert len(preds) == len(gts)
    hits = 0
    for q, a in preds.items():
        gt_answers = gts[q]
        hits+= int(eval_utils.metric_max_over_ground_truths(
            metric_fn=exact_match_score, prediction=a, ground_truths=gt_answers))
    acc = hits*100./len(preds)
    return acc

In [14]:
def evaluate_model(S):
    fused_predictions = dict()
#     if not S.shape[0] == len(extractive_reader_predictions):
#         print(S.shape)
#         print(len(extractive_reader_predictions))
    assert S.shape[0] == len(extractive_reader_predictions)
    for i, (e_p, ranked_p, reranked_p) in enumerate(zip(extractive_reader_predictions.values(), ranked_predictions, reranked_predictions)):
#         if not e_p['raw_question'] == ranked_p['question'] == reranked_p['question']:
#             print(e_p['raw_question'])
#             print(ranked_p['question'])
#             print(reranked_p['question'])
        assert e_p['raw_question'] == ranked_p['question'] == reranked_p['question']
        fused_scores= S[i].tolist()
        def argmax(l):
            f = lambda i: l[i]
            return max(range(len(l)), key=f)
        fused_predictions[e_p["raw_question"]] =  e_p["answers"][argmax(fused_scores)]
    return evaluate(fused_predictions,correct)

In [25]:
import torch.nn.functional as F
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
def run_training():
    STEPS=100
    model =  ConstrainedLR()
    model = model.train()
    opt = torch.optim.SGD(model.parameters(), lr=0.3)
    scheduler = get_linear_schedule_with_warmup(
                    opt,
                    num_warmup_steps=10,
                    num_training_steps=STEPS
                )

    best_acc= 0
    best_r = None
    iterator = tqdm(range(STEPS))
    for i in iterator:
        logits = model(X, Y, Z)
        l_list = F.cross_entropy(logits, labels, reduction='none')
        l = l_list.mean()
        l.backward()
        opt.step()
        scheduler.step()
        opt.zero_grad()
        if i % 1 == 0:
            r = { k: v.tolist() for k,v in dict(model.named_parameters()).items()}
            model.eval()
            with torch.no_grad():
                acc = evaluate_model(model(X_test, Y_test, Z_test))
            model.train()
            if acc>best_acc:
                iterator.set_description(str(acc))
                best_acc = acc
                best_r = r
    return best_acc, best_r
total_best_acc, total_best_r = 0.,None
for _ in range(20):
    acc,r = run_training()
    if acc>total_best_acc:
        print(r)
        total_best_acc=acc
        total_best_r=r

46.5:   2%|▏         | 2/100 [00:00<00:06, 14.54it/s]              

{'a': [0.4434039890766144], 'b': [0.08466459810733795], 'c': [0.019445449113845825], 'bias': [1.1867865324020386]}


45.333333333333336:   2%|▏         | 2/100 [00:00<00:06, 14.49it/s]

{'a': [0.42799437046051025], 'b': [0.025308959186077118], 'c': [0.09631370007991791], 'bias': [0.9029167890548706]}


47.44444444444444: 100%|██████████| 100/100 [00:06<00:00, 16.33it/s]


e.g.
```
{'a': [0.9799807071685791], 'b': [0.23300687968730927], 'c': [0.1416887491941452], 'bias': [0.802649199962616]}
```


In [16]:
print(f"Best accuracy: {total_best_acc}")
print(f"Best parameters:\n{total_best_r}")

Best accuracy: 47.666666666666664
Best parameters:
{'a': [0.44447702169418335], 'b': [0.032241690903902054], 'c': [0.11667594313621521], 'bias': [0.5796049237251282]}


In [17]:
# Use only extractive reader
model =  ConstrainedLR()
model = model.eval()
model.a[0] = 1.
model.b[0] = 0.
model.c[0] = 0.
model.bias[0] = 0.
evaluate_model(model(X_test, Y_test, Z_test))

46.55555555555556

In [18]:
# Use only ranker
model =  ConstrainedLR()
model = model.eval()
model.a[0] = 0.
model.b[0] = 1.
model.c[0] = 0.
model.bias[0] = 0.
evaluate_model(model(X_test, Y_test, Z_test))

41.111111111111114

In [19]:
# Use only reranker
model =  ConstrainedLR()
model = model.eval()
model.a[0] = 0.
model.b[0] = 0.
model.c[0] = 1.
model.bias[0] = 0.
evaluate_model(model(X_test, Y_test, Z_test))

42.27777777777778

In [20]:
# Use nothing (sanity check)
model =  ConstrainedLR()
model = model.eval()
model.a[0] = 0.
model.b[0] = 0.
model.c[0] = 0.
model.bias[0] = 0.
evaluate_model(model(X_test, Y_test, Z_test))

42.833333333333336

In [21]:
S = model(X_test, Y_test, Z_test)